# Transformer (Ko-En) 번역기 v2.0
---
### 프로젝트 목표
한국어-영어 번역을 위한 트랜스포머 모델에 메캅 형태소 분석 추가

**주요 변경 사항 (v1.3 대비):**
1. 데이터 증강: NLLB 모델을 활용한 역번역(Back-translation) 기법을 도입하여 학습 데이터의 양과 질을 대폭 향상.
2. 토크나이저: 한국어 토큰화 시, `Mecab` 형태소 분석기를 `SentencePiece` 이전에 적용하여 언어적 특성 반영을 강화.
3. 훈련 방식 업그레이드:
    - 옵티마이저: Adam -> AdamW로 변경하고 `weight_decay`를 적용하여 정규화 성능 개선.
    - 손실 함수: `Label Smoothing`을 적용하여 모델의 과신을 방지하고 일반화 성능 향상.
    - 학습률 스케줄러: `역제곱근 감쇠` 방식 (All You Need is Attention 논문 방식)-> '코사인 어닐링' 방식으로 변경하여 더 안정적인 수렴 유도.
4. 평가 방식: 빔 서치(Beam Search) 디코딩을 기본으로 사용하고, BLEU, METEOR, ROUGE, BERTScore를 모두 측정하는 종합 평가 파이프라인 구축.

## 1. 라이브러리 설치 및 임포트

In [ ]:
!pip install sentencepiece
!pip install gensim
!pip install nltk
!pip install konlpy
!pip install transformers
!pip install rouge-score bert-score
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab/
!bash install_mecab-ko_on_colab_light_220429.sh

%cd ..

In [ ]:
import os
import re
import math
import time
import random
import locale

# 데이터 처리 및 연산
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR, LinearLR, SequentialLR
from torch.utils.data import Dataset, DataLoader
from transformers import pipeline

# 자연어 처리(NLP) 및 머신러닝
import sentencepiece as spm
from gensim.models import KeyedVectors
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# 시각화 및 진행률 표시
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from tqdm.notebook import tqdm

In [ ]:
def set_seed(seed):
    """모든 랜덤 시드를 고정하여 재현성을 보장합니다."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed) # for multi-GPU

# 사용할 시드 값 설정
SEED = 14
set_seed(SEED)

print(f"Random seed set to {SEED}")

## 2. 하이퍼파라미터 및 설정

In [ ]:
# Model Hyperparameters
SRC_VOCAB_SIZE = 20000
TGT_VOCAB_SIZE = 20000
D_MODEL = 512
N_LAYERS = 4
N_HEADS = 8
D_FF = 4096
DROPOUT = 0.15
MAX_LEN = 50

# Training Hyperparameters
BATCH_SIZE = 64
EPOCHS = 30
EARLY_STOPPING_PATIENCE = 3
CHECKPOINT_PATH = "transformer-2.0-checkpoint.pth"

device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(f'Using device: {device}')

## 3. 데이터 준비 및 전처리 & 증강 (역번역)

In [ ]:
# 1. 데이터 경로 설정
data_dir = 'data'
train_kor_path = os.path.join(data_dir, 'korean-english-park.train.ko')
train_eng_path = os.path.join(data_dir, 'korean-english-park.train.en')
dev_kor_path = os.path.join(data_dir, 'korean-english-park.dev.ko')
dev_eng_path = os.path.join(data_dir, 'korean-english-park.dev.en')
test_kor_path = os.path.join(data_dir, 'korean-english-park.test.ko')
test_eng_path = os.path.join(data_dir, 'korean-english-park.test.en')

# 2. 원본 데이터 로딩
with open(train_kor_path, "r", encoding='utf-8') as f: train_kor_raw = f.read().splitlines()
with open(train_eng_path, "r", encoding='utf-8') as f: train_eng_raw = f.read().splitlines()
with open(dev_kor_path, "r", encoding='utf-8') as f: dev_kor_raw = f.read().splitlines()
with open(dev_eng_path, "r", encoding='utf-8') as f: dev_eng_raw = f.read().splitlines()
with open(test_kor_path, "r", encoding='utf-8') as f: test_kor_raw = f.read().splitlines()
with open(test_eng_path, "r", encoding='utf-8') as f: test_eng_raw = f.read().splitlines()

print(f"Train: {len(train_kor_raw)}, Dev: {len(dev_kor_raw)}, Test: {len(test_kor_raw)}")

In [ ]:
# NLLB 모델 로드 (GPU 사용)
translator = pipeline(
    'translation',
    model='facebook/nllb-200-distilled-600M',
    device=0 if torch.cuda.is_available() else -1
)

def back_translate_nllb_optimized(sentences, chunk_size=1000, batch_size=32):
    """
    역번역을 진행하면서 진행 상황을 출력하고, 효율성을 높이는 함수.
    Args:
        sentences (list): 역번역할 한국어 문장 리스트.
        chunk_size (int): 진행 상황을 출력할 단위.
        batch_size (int): 모델 추론 시 사용할 배치 크기 (GPU 효율성 향상).
    Returns:
        list: 역번역된 문장 리스트.
    """
    augmented_sentences = []
    total_sentences = len(sentences)
    
    print(f"총 {total_sentences}개의 문장을 역번역합니다...")
    
    start_time = time.time()
    
    # 데이터를 청크 단위로 분할하여 처리
    for i in range(0, total_sentences, chunk_size):
        chunk_start = i
        chunk_end = min(i + chunk_size, total_sentences)
        
        # 한국어 -> 영어 번역 (배치 크기 적용)
        translated_to_en = [
            result['translation_text'] for result in translator(sentences[chunk_start:chunk_end], 
                                                               src_lang='kor_Hang', 
                                                               tgt_lang='eng_Latn',
                                                               batch_size=batch_size) # 배치 크기 인자 추가
        ]
        
        # 영어 -> 한국어 역번역 (배치 크기 적용)
        back_translated_to_ko = [
            result['translation_text'] for result in translator(translated_to_en, 
                                                               src_lang='eng_Latn', 
                                                               tgt_lang='kor_Hang',
                                                               batch_size=batch_size) # 배치 크기 인자 추가
        ]
        
        augmented_sentences.extend(back_translated_to_ko)
        
        elapsed_time = time.time() - start_time
        print(f"[{chunk_end}/{total_sentences}] 문장 처리 완료 | 경과 시간: {elapsed_time:.2f}초")

    return augmented_sentences

# 최적화된 함수 사용
print("\n한국어 학습 데이터 역번역을 시작합니다...")
augmented_train_kor_raw = back_translate_nllb_optimized(train_kor_raw)
print("데이터 증강 완료")

# 원본 데이터와 증강된 데이터를 합치기
combined_train_kor_raw = train_kor_raw + augmented_train_kor_raw
combined_train_eng_raw = train_eng_raw + train_eng_raw

print(f"최종 학습 데이터 크기: 한국어 {len(combined_train_kor_raw)}, 영어 {len(combined_train_eng_raw)}")

In [ ]:
# 3. 데이터 정제 및 전처리
def preprocess_sentence(sentence):
    """구두점, 특수문자 등 불필요한 부분을 제거하고 소문자로 변환합니다."""
    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z가-힣?.!,]+", " ", sentence)
    sentence = sentence.strip()
    return sentence

def clean_and_preprocess_corpus(kor_raw, eng_raw):
    """문장 쌍의 중복을 제거하고 각 문장을 전처리합니다."""
    # 1. zip으로 문장 쌍 생성 후 set으로 중복 제거
    cleaned_pairs = list(set(zip(kor_raw, eng_raw)))

    # 2. 각 문장 전처리
    kor_corpus, eng_corpus = [], []
    for kor, eng in cleaned_pairs:
        kor_corpus.append(preprocess_sentence(kor))
        eng_corpus.append(preprocess_sentence(eng))

    return kor_corpus, eng_corpus

# 각 데이터셋에 대해 정제 및 전처리 수행
train_kor_corpus, train_eng_corpus = clean_and_preprocess_corpus(combined_train_kor_raw, combined_train_eng_raw)
dev_kor_corpus, dev_eng_corpus = clean_and_preprocess_corpus(dev_kor_raw, dev_eng_raw)
test_kor_corpus, test_eng_corpus = clean_and_preprocess_corpus(test_kor_raw, test_eng_raw)

print(f"Train: {len(train_kor_corpus)}, Dev: {len(dev_kor_corpus)}, Test: {len(test_kor_corpus)}")

In [ ]:
from konlpy.tag import Mecab

# 1. Mecab 초기화
mecab = Mecab()
def mecab_tokenize_corpus(corpus):
    mecab_corpus = []
    for sentence in corpus:
        # 형태소 분리 후 공백으로 join
        morphs = mecab.morphs(sentence)
        mecab_corpus.append(" ".join(morphs))
    return mecab_corpus

# 2. 한국어 데이터셋 Mecab 처리
train_kor_mecab = mecab_tokenize_corpus(train_kor_corpus)
dev_kor_mecab   = mecab_tokenize_corpus(dev_kor_corpus)
test_kor_mecab  = mecab_tokenize_corpus(test_kor_corpus)

print("Before:", train_kor_corpus[0])
print("After :", train_kor_mecab[0])

In [ ]:
def generate_tokenizer(corpus, vocab_size, lang, pad_id=0, bos_id=1, eos_id=2, unk_id=3):
    file = f'./{lang}_corpus.txt'
    model_prefix = f'{lang}_spm'
    with open(file, 'w', encoding='utf-8') as f:
        for row in corpus:
            f.write(str(row) + '\n')
    spm.SentencePieceTrainer.Train(
        f'--input={file} --model_prefix={model_prefix} --vocab_size={vocab_size}' + 
        f' --pad_id={pad_id} --bos_id={bos_id} --eos_id={eos_id} --unk_id={unk_id}'
    )
    tokenizer = spm.SentencePieceProcessor()
    tokenizer.Load(f'{model_prefix}.model')
    return tokenizer

ko_tokenizer = generate_tokenizer(train_kor_mecab, SRC_VOCAB_SIZE, "ko")
en_tokenizer = generate_tokenizer(train_eng_corpus, TGT_VOCAB_SIZE, "en")

## 4. 데이터셋 및 DataLoader 구축

In [ ]:
class TranslationDataset(Dataset):
    def __init__(self, src_corpus, tgt_corpus, src_tokenizer, tgt_tokenizer):
        self.src_corpus = src_corpus
        self.tgt_corpus = tgt_corpus
        self.src_tokenizer = src_tokenizer
        self.tgt_tokenizer = tgt_tokenizer

    def __len__(self):
        return len(self.src_corpus)

    def __getitem__(self, idx):
        src = self.src_tokenizer.encode_as_ids(self.src_corpus[idx])
        tgt = self.tgt_tokenizer.encode_as_ids(self.tgt_corpus[idx])

        # 텐서의 데이터 타입을 torch.long으로 명시적으로 지정합니다.
        return torch.tensor(src, dtype=torch.long), torch.tensor(tgt, dtype=torch.long)

def collate_fn(batch):
    """배치 내의 시퀀스들을 패딩하여 동일한 길이로 만듭니다."""
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(src_sample)
        tgt_batch.append(tgt_sample)

    src_padded = torch.nn.utils.rnn.pad_sequence(src_batch, batch_first=True, padding_value=ko_tokenizer.pad_id())
    tgt_padded = torch.nn.utils.rnn.pad_sequence(tgt_batch, batch_first=True, padding_value=en_tokenizer.pad_id())
    return src_padded, tgt_padded

# Dataset 및 DataLoader 인스턴스 생성
train_dataset = TranslationDataset(train_kor_mecab, train_eng_corpus, ko_tokenizer, en_tokenizer)
valid_dataset = TranslationDataset(dev_kor_mecab, dev_eng_corpus, ko_tokenizer, en_tokenizer)
test_dataset = TranslationDataset(test_kor_mecab, test_eng_corpus, ko_tokenizer, en_tokenizer)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn, num_workers=4)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, num_workers=4)

print(f"Number of batches in train_loader: {len(train_loader)}")
print(f"Number of batches in valid_loader: {len(valid_loader)}")
print(f"Number of batches in test_loader: {len(test_loader)}")

## 5. 트랜스포머 모델 정의

In [ ]:
class PositionalEncoding(nn.Module):
    """
    입력 임베딩에 위치 정보를 추가하는 클래스입니다.
    Transformer 모델은 순서 정보가 없으므로, 토큰의 위치를 알려주기 위해 sin/cos 함수를 사용합니다.
    이 방식은 고정 위치 인코딩으로, 학습되지 않는 파라미터(buffer)로 등록됩니다.
    """
    def __init__(self, emb_size: int, dropout: float, maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        # sin/cos 함수에 사용할 div_term 계산: 주파수 조절을 위한 값
        div_term = torch.exp(torch.arange(0, emb_size, 2) * (-math.log(10000.0) / emb_size))
        # 각 위치(0~maxlen)에 대한 인덱스 생성
        position = torch.arange(maxlen).unsqueeze(1)
        # 위치 임베딩 행렬 초기화 (maxlen, emb_size)
        pos_embedding = torch.zeros(maxlen, emb_size)
        # 짝수 인덱스: sin 함수 적용
        pos_embedding[:, 0::2] = torch.sin(position * div_term)
        # 홀수 인덱스: cos 함수 적용
        pos_embedding[:, 1::2] = torch.cos(position * div_term)
        # 배치 차원 추가 (1, maxlen, emb_size)
        pos_embedding = pos_embedding.unsqueeze(0)
        self.dropout = nn.Dropout(dropout)
        # 학습되지 않는 파라미터로 등록
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding):
        """
        Args:
            token_embedding: (batch_size, seq_len, emb_size)
        Returns:
            token_embedding + pos_embedding: 위치 정보가 더해진 임베딩
        """
        return self.dropout(token_embedding + self.pos_embedding[:, :token_embedding.size(1), :])

class MultiHeadAttention(nn.Module):
    """
    다중 헤드 어텐션 메커니즘을 구현한 클래스.
    쿼리, 키, 값 행렬을 여러 헤드로 분할하여 병렬로 어텐션을 계산하고, 결과를 결합합니다.
    """
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        self.depth = d_model // num_heads  # 각 헤드의 차원
        # 쿼리, 키, 값 행렬을 위한 선형 변환 레이어
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        # 최종 출력 선형 변환 레이어
        self.linear = nn.Linear(d_model, d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        """
        스케일드 닷-프로덕트 어텐션 계산.
        Args:
            Q: 쿼리 행렬
            K: 키 행렬
            V: 값 행렬
            mask: 어텐션 마스크 (선택적)
        Returns:
            out: 어텐션 가중치 적용된 값 행렬
            attentions: 어텐션 가중치 행렬
        """
        d_k = Q.size(-1)
        QK = torch.matmul(Q, K.transpose(-1, -2))  # QK^T 계산
        scaled_qk = QK / math.sqrt(d_k)  # 스케일링
        if mask is not None:
            scaled_qk += (mask * -1e9)  # 마스크 적용 (매우 작은 값 더하기)
        attentions = nn.Softmax(dim=-1)(scaled_qk)  # 소프트맥스 적용
        out = torch.matmul(attentions, V)  # 가중치 적용
        return out, attentions

    def split_heads(self, x):
        """
        입력 텐서를 여러 헤드로 분할.
        Args:
            x: (batch_size, seq_len, d_model)
        Returns:
            x: (batch_size, num_heads, seq_len, depth)
        """
        bsz, seq_len, _ = x.size()
        x = x.view(bsz, seq_len, self.num_heads, self.depth)
        return x.permute(0, 2, 1, 3)  # 차원 재배치

    def combine_heads(self, x):
        """
        분할된 헤드를 다시 결합.
        Args:
            x: (batch_size, num_heads, seq_len, depth)
        Returns:
            x: (batch_size, seq_len, d_model)
        """
        bsz, _, seq_len, _ = x.size()
        x = x.permute(0, 2, 1, 3).contiguous()
        return x.view(bsz, seq_len, self.d_model)

    def forward(self, Q, K, V, mask=None):
        """
        Args:
            Q: 쿼리 입력 (batch_size, seq_len, d_model)
            K: 키 입력
            V: 값 입력
            mask: 어텐션 마스크
        Returns:
            out: 어텐션 적용된 출력
            attention_weights: 어텐션 가중치
        """
        # 헤드 분할 후 어텐션 계산
        WQ = self.split_heads(self.W_q(Q))
        WK = self.split_heads(self.W_k(K))
        WV = self.split_heads(self.W_v(V))
        out, attention_weights = self.scaled_dot_product_attention(WQ, WK, WV, mask)
        # 헤드 결합 후 선형 변환
        out = self.combine_heads(out)
        out = self.linear(out)
        return out, attention_weights

class PoswiseFeedForwardNet(nn.Module):
    """
    포지션 와이즈 피드포워드 네트워크.
    각 위치별로 독립적으로 적용되는 2층 완전 연결 네트워크 (ReLU 활성화 함수 사용).
    """
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)  # 첫 번째 레이어 (차원 확장)
        self.fc2 = nn.Linear(d_ff, d_model)  # 두 번째 레이어 (원래 차원으로 복원)
        self.relu = nn.ReLU()

    def forward(self, x):
        """
        Args:
            x: (batch_size, seq_len, d_model)
        Returns:
            x: (batch_size, seq_len, d_model)
        """
        return self.fc2(self.relu(self.fc1(x)))

class EncoderLayer(nn.Module):
    """
    인코더의 단일 레이어.
    셀프 어텐션과 피드포워드 네트워크를 포함하며, 레이어 정규화와 드롭아웃을 적용합니다.
    """
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)
        self.norm_1 = nn.LayerNorm(d_model, eps=1e-6)  # 첫 번째 정규화
        self.norm_2 = nn.LayerNorm(d_model, eps=1e-6)  # 두 번째 정규화
        self.do = nn.Dropout(dropout)

    def forward(self, x, mask):
        """
        Args:
            x: 입력 텐서
            mask: 패딩 마스크
        Returns:
            out: 출력 텐서
            enc_attn: 셀프 어텐션 가중치
        """
        residual = x
        # 셀프 어텐션 + 드롭아웃 + 잔차 연결
        out, enc_attn = self.enc_self_attn(self.norm_1(x), self.norm_1(x), self.norm_1(x), mask)
        out = self.do(out) + residual
        residual = out
        # 피드포워드 네트워크 + 드롭아웃 + 잔차 연결
        out = self.ffn(self.norm_2(out))
        out = self.do(out) + residual
        return out, enc_attn

class DecoderLayer(nn.Module):
    """
    디코더의 단일 레이어.
    셀프 어텐션, 인코더-디코더 어텐션, 피드포워드 네트워크를 포함합니다.
    """
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.dec_self_attn = MultiHeadAttention(d_model, num_heads)  # 셀프 어텐션
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)  # 인코더-디코더 어텐션
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)
        self.norm_1 = nn.LayerNorm(d_model, eps=1e-6)
        self.norm_2 = nn.LayerNorm(d_model, eps=1e-6)
        self.norm_3 = nn.LayerNorm(d_model, eps=1e-6)
        self.do = nn.Dropout(dropout)

    def forward(self, x, enc_out, dec_enc_mask, padding_mask):
        """
        Args:
            x: 디코더 입력
            enc_out: 인코더 출력
            dec_enc_mask: 디코더-인코더 어텐션 마스크
            padding_mask: 패딩 마스크
        Returns:
            out: 출력 텐서
            dec_attn: 셀프 어텐션 가중치
            dec_enc_attn: 인코더-디코더 어텐션 가중치
        """
        residual = x
        # 셀프 어텐션 (look-ahead 마스크 적용)
        out, dec_attn = self.dec_self_attn(self.norm_1(x), self.norm_1(x), self.norm_1(x), mask=padding_mask)
        out = self.do(out) + residual
        residual = out
        # 인코더-디코더 어텐션
        out, dec_enc_attn = self.enc_dec_attn(self.norm_2(out), enc_out, enc_out, mask=dec_enc_mask)
        out = self.do(out) + residual
        residual = out
        # 피드포워드 네트워크
        out = self.ffn(self.norm_3(out))
        out = self.do(out) + residual
        return out, dec_attn, dec_enc_attn

class Encoder(nn.Module):
    """
    인코더 전체 구조.
    임베딩 레이어, 위치 인코딩, 여러 개의 인코더 레이어로 구성됩니다.
    """
    def __init__(self, n_layers, d_model, n_heads, d_ff, dropout, vocab_size):
        super(Encoder, self).__init__()
        self.d_model = d_model
        self.embedding = nn.Embedding(vocab_size, d_model)  # 토큰 임베딩
        self.pos_encoding = PositionalEncoding(d_model, dropout)  # 위치 인코딩
        self.enc_layers = nn.ModuleList([EncoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(n_layers)])

    def forward(self, x, mask):
        """
        Args:
            x: 입력 시퀀스 (batch_size, seq_len)
            mask: 패딩 마스크
        Returns:
            out: 인코더 출력
            enc_attns: 각 레이어의 어텐션 가중치 리스트
        """
        out = self.embedding(x) * math.sqrt(self.d_model)  # 임베딩 스케일링
        out = self.pos_encoding(out)  # 위치 인코딩 추가
        enc_attns = []
        for layer in self.enc_layers:
            out, enc_attn = layer(out, mask)
            enc_attns.append(enc_attn)
        return out, enc_attns

class Decoder(nn.Module):
    """
    디코더 전체 구조.
    임베딩 레이어, 위치 인코딩, 여러 개의 디코더 레이어로 구성됩니다.
    """
    def __init__(self, n_layers, d_model, n_heads, d_ff, dropout, vocab_size):
        super(Decoder, self).__init__()
        self.d_model = d_model
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding(d_model, dropout)
        self.dec_layers = nn.ModuleList([DecoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(n_layers)])

    def forward(self, x, enc_out, dec_enc_mask, padding_mask):
        """
        Args:
            x: 디코더 입력 시퀀스
            enc_out: 인코더 출력
            dec_enc_mask: 디코더-인코더 어텐션 마스크
            padding_mask: 패딩 마스크
        Returns:
            out: 디코더 출력
            dec_attns: 셀프 어텐션 가중치 리스트
            dec_enc_attns: 인코더-디코더 어텐션 가중치 리스트
        """
        out = self.embedding(x) * math.sqrt(self.d_model)
        out = self.pos_encoding(out)
        dec_attns, dec_enc_attns = [], []
        for layer in self.dec_layers:
            out, dec_attn, dec_enc_attn = layer(out, enc_out, dec_enc_mask, padding_mask)
            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)
        return out, dec_attns, dec_enc_attns

class Transformer(nn.Module):
    """
    전체 Transformer 모델.
    인코더와 디코더를 연결하고, 최종 출력 레이어를 포함합니다.
    """
    def __init__(self, n_layers, d_model, n_heads, d_ff, src_vocab_size, tgt_vocab_size, dropout):
        super(Transformer, self).__init__()
        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout, src_vocab_size)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout, tgt_vocab_size)
        self.fc = nn.Linear(d_model, tgt_vocab_size)  # 최종 출력 레이어

    def forward(self, src, tgt):
        """
        Args:
            src: 소스 시퀀스 (batch_size, src_seq_len)
            tgt: 타겟 시퀀스 (batch_size, tgt_seq_len)
        Returns:
            logits: 최종 예측 로짓 (batch_size, tgt_seq_len, tgt_vocab_size)
            enc_attns: 인코더 어텐션 가중치 리스트
            dec_attns: 디코더 셀프 어텐션 가중치 리스트
            dec_enc_attns: 디코더-인코더 어텐션 가중치 리스트
        """
        # 마스크 생성
        src_mask = (src == ko_tokenizer.pad_id()).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt == en_tokenizer.pad_id()).unsqueeze(1).unsqueeze(2)
        lookahead_mask = torch.triu(torch.ones(tgt.shape[1], tgt.shape[1]), diagonal=1).bool().to(device)
        tgt_mask = tgt_mask | lookahead_mask
        # 인코더/디코더 순전파
        enc_out, enc_attns = self.encoder(src, src_mask)
        dec_out, dec_attns, dec_enc_attns = self.decoder(tgt, enc_out, src_mask, tgt_mask)
        logits = self.fc(dec_out)
        return logits, enc_attns, dec_attns, dec_enc_attns

## 6. 학습 설정

In [ ]:
class CustomLearningRateScheduler:
    """
    "Attention Is All You Need" 논문에서 제안된 custom learning rate scheduler.
    Warm-up 기간 동안 학습률을 선형적으로 증가시킨 후, step 수의 역제곱근에 비례하여 감소시킵니다.
    """
    def __init__(self, optimizer, d_model, warmup_steps=4000):
        self.optimizer = optimizer
        self.d_model = d_model
        self.warmup_steps = warmup_steps
        self.num_steps = 0

    def step(self):
        """학습률을 업데이트합니다."""
        self.num_steps += 1
        lr = self._get_lr()
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = lr

    def _get_lr(self):
        """학습률을 계산합니다."""
        step = self.num_steps
        # 수식: lrate = d_model**(-0.5) * min(step**(-0.5), step * warmup_steps**(-1.5))
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)
        return (self.d_model ** -0.5) * min(arg1, arg2)

    def state_dict(self):
        """스케줄러의 상태를 반환합니다."""
        return {'num_steps': self.num_steps}

    def load_state_dict(self, state_dict):
        """스케줄러의 상태를 불러옵니다."""
        self.num_steps = state_dict['num_steps']

In [ ]:
# 모델, 손실 함수 초기화 (Label Smoothing 포함)
model = Transformer(N_LAYERS, D_MODEL, N_HEADS, D_FF, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, DROPOUT).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=ko_tokenizer.pad_id(), label_smoothing=0.1)

# 옵티마이저: AdamW 사용 및 최대 학습률(peak_lr) 설정
peak_lr = 5e-5
optimizer = optim.AdamW(model.parameters(), lr=peak_lr, betas=(0.9, 0.98), eps=1e-9, weight_decay=0.01)

warmup_steps = 4000
# 전체 훈련 스텝 계산 (CosineAnnealingLR의 T_max에 필요)
total_training_steps = len(train_loader) * EPOCHS

# 1. Warmup 스케줄러 (LinearLR)
# warmup_steps 동안 학습률을 0에서 peak_lr까지 선형적으로 증가
warmup_scheduler = LinearLR(optimizer, start_factor=0.001, total_iters=warmup_steps)

# 2. Main 스케줄러 (CosineAnnealingLR)
# Warmup 이후, 남은 스텝 동안 학습률을 코사인 곡선을 따라 부드럽게 감소시킵니다.
main_scheduler = CosineAnnealingLR(optimizer, T_max=total_training_steps - warmup_steps, eta_min=1e-6)

# 3. 두 스케줄러를 순차적으로 연결 (SequentialLR)
# warmup_steps 이전까지는 warmup_scheduler를, 이후에는 main_scheduler를 사용
scheduler = SequentialLR(
    optimizer,
    schedulers=[warmup_scheduler, main_scheduler],
    milestones=[warmup_steps]
)

print("훈련 설정이 'Warmup + Cosine Annealing' 스케줄러로 업그레이드되었습니다.")
print(f"최대 학습률(Peak LR): {peak_lr}, 웜업 스텝: {warmup_steps}")

# 체크포인트 불러오기
start_epoch = 0
best_valid_loss = float('inf')

if os.path.exists(CHECKPOINT_PATH):
    print(f"체크포인트를 불러옵니다: {CHECKPOINT_PATH}")
    checkpoint = torch.load(CHECKPOINT_PATH)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    start_epoch = checkpoint['epoch']
    best_valid_loss = checkpoint['best_valid_loss']
    print(f"체크포인트 로드 완료. Epoch {start_epoch + 1}부터 훈련을 재개합니다.")
    model.to(device) # 모델을 device로 이동
else:
    print("체크포인트가 없습니다. 처음부터 훈련을 시작합니다.")

## 7. 학습 및 검증

In [ ]:
def train(model, iterator, optimizer, scheduler, criterion, clip):
    model.train()
    epoch_loss = 0
    progress_bar = tqdm(iterator, desc="Training", mininterval=0.5, leave=False)

    for i, batch in enumerate(progress_bar):
        src = batch[0].to(device)
        tgt = batch[1].to(device)

        optimizer.zero_grad()

        output, _, _, _ = model(src, tgt[:,:-1])
        output_dim = output.shape[-1]
        output = output.contiguous().view(-1, output_dim)
        tgt = tgt[:,1:].contiguous().view(-1)

        loss = criterion(output, tgt)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        scheduler.step()
        optimizer.step()

        epoch_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item(), lr=scheduler.optimizer.param_groups[0]['lr'])

    return epoch_loss / len(iterator)

def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    progress_bar = tqdm(iterator, desc="Evaluating", mininterval=0.5, leave=False)

    with torch.no_grad():
        for i, batch in enumerate(progress_bar):
            src = batch[0].to(device)
            tgt = batch[1].to(device)

            output, _, _, _ = model(src, tgt[:,:-1])
            output_dim = output.shape[-1]
            output = output.contiguous().view(-1, output_dim)
            tgt = tgt[:,1:].contiguous().view(-1)

            loss = criterion(output, tgt)
            epoch_loss += loss.item()
            progress_bar.set_postfix(loss=loss.item())

    return epoch_loss / len(iterator)

# --- 학습 루프 ---
early_stopping_counter = 0

for epoch in range(start_epoch, EPOCHS):
    start_time = time.time()

    print(f"Epoch {epoch+1:02} / {EPOCHS:02}")

    train_loss = train(model, train_loader, optimizer, scheduler, criterion, 1)
    valid_loss = evaluate(model, valid_loader, criterion)

    end_time = time.time()
    epoch_mins, epoch_secs = divmod(end_time - start_time, 60)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        early_stopping_counter = 0
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'best_valid_loss': best_valid_loss,
        }, CHECKPOINT_PATH)
        print(f"Validation loss improved. Checkpoint saved to {CHECKPOINT_PATH}")
    else:
        early_stopping_counter += 1
        print(f"Validation loss did not improve. Counter: {early_stopping_counter}/{EARLY_STOPPING_PATIENCE}")

    print(f'Time: {epoch_mins:.0f}m {epoch_secs:.0f}s')
    print(f'	Train Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'	 Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
    print("-" * 30)

    if early_stopping_counter >= EARLY_STOPPING_PATIENCE:
        print(f"조기 종료: {EARLY_STOPPING_PATIENCE} 에폭 동안 검증 손실이 개선되지 않았습니다.")
        break

## 8. 번역 (greedy Search, Beam Search)

In [ ]:
# 1. 새로운 모델 객체를 만들고 저장된 가중치를 불러옴
inference_model = Transformer(N_LAYERS, D_MODEL, N_HEADS, D_FF, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, DROPOUT).to(device)

# 체크포인트는 딕셔너리 형태이므로, model_state_dict를 직접 로드해야 합니다.
checkpoint = torch.load(CHECKPOINT_PATH, map_location=device)
inference_model.load_state_dict(checkpoint['model_state_dict'])

# 2. 모델을 평가 모드로 설정
inference_model.eval()

In [ ]:
def translate_sentence(sentence, src_tokenizer, tgt_tokenizer, model, device, max_len=50):

    src_tokens = src_tokenizer.encode_as_ids(sentence)
    src_tensor = torch.LongTensor(src_tokens).unsqueeze(0).to(device)

    tgt_tokens = [tgt_tokenizer.bos_id()]

    for i in range(max_len):
        tgt_tensor = torch.LongTensor(tgt_tokens).unsqueeze(0).to(device)

        with torch.no_grad():
            output, _, _, dec_enc_attns = model(src_tensor, tgt_tensor)

        pred_token = output.argmax(2)[:,-1].item()
        tgt_tokens.append(pred_token)

        if pred_token == tgt_tokenizer.eos_id():
            break

    tgt_sentence = tgt_tokenizer.decode_ids(tgt_tokens)
    return tgt_sentence, dec_enc_attns

In [ ]:
def translate_sentence_beam_search(sentence, src_tokenizer, tgt_tokenizer, model, device, max_len=50, beam_size=5):
    """
    빔 서치(Beam Search)를 사용하여 문장을 번역하고, 최종 번역문에 대한 어텐션 맵을 반환하는 함수입니다.
    """
    model.eval()

    src_tokens = src_tokenizer.encode_as_ids(sentence)
    src_tensor = torch.LongTensor(src_tokens).unsqueeze(0).to(device)

    beams = [(torch.LongTensor([tgt_tokenizer.bos_id()]).to(device), 0)]
    completed_hypotheses = []

    for _ in range(max_len):
        new_beams = []
        for seq, score in beams:
            if seq[-1].item() == tgt_tokenizer.eos_id():
                completed_hypotheses.append((seq, score))
                continue

            with torch.no_grad():
                output, _, _, _ = model(src_tensor, seq.unsqueeze(0))

            next_token_logits = output[:, -1, :]
            next_token_log_probs = torch.log_softmax(next_token_logits, dim=-1)
            top_next_tokens = torch.topk(next_token_log_probs, beam_size, dim=-1)

            for i in range(beam_size):
                token_id = top_next_tokens.indices[0][i].item()
                log_prob = top_next_tokens.values[0][i].item()

                new_seq = torch.cat([seq, torch.LongTensor([token_id]).to(device)])
                new_score = score + log_prob
                new_beams.append((new_seq, new_score))

        if not new_beams:
            break

        beams = sorted(new_beams, key=lambda x: x[1], reverse=True)[:beam_size]

        if all(b[0][-1].item() == tgt_tokenizer.eos_id() for b in beams):
            completed_hypotheses.extend(beams)
            break

    if not completed_hypotheses:
        completed_hypotheses.extend(beams)

    best_hypothesis = sorted(completed_hypotheses, key=lambda x: x[1] / len(x[0]), reverse=True)[0]
    best_sequence = best_hypothesis[0]

    translated_sentence = tgt_tokenizer.decode_ids(best_sequence.tolist())

    # 최종 선택된 시퀀스에 대한 어텐션 맵을 얻기 위해 모델을 한 번 더 실행
    with torch.no_grad():
        # </s> 토큰은 어텐션 계산에 필요 없으므로, 있다면 제외
        input_seq = best_sequence.unsqueeze(0)
        if input_seq[0, -1].item() == tgt_tokenizer.eos_id():
            input_seq = input_seq[:, :-1]

        _, _, _, final_attentions = model(src_tensor, input_seq)

    return translated_sentence, final_attentions


In [ ]:
# 번역할 문장 선택
example_idx = 1
src = test_kor_corpus[example_idx]
trg = test_eng_corpus[example_idx]

# 빔 서치로 번역 실행 (beam_size=5)
beam_translation, beam_attention = translate_sentence_beam_search(src, ko_tokenizer, en_tokenizer, inference_model, device, beam_size=5)

print(f'src = {src}')
print(f'trg = {trg}')
print(f'predicted trg (beam search) = {beam_translation}')

# 기존 Greedy 방식과 비교
greedy_translation, _ = translate_sentence(src, ko_tokenizer, en_tokenizer, inference_model, device)
print(f'predicted trg (greedy)      = {greedy_translation}')


## 9. 어텐션 시각화

In [ ]:
def display_attention(sentence, translation, attention, n_heads=8, n_rows=4, n_cols=2):
    """어텐션 맵을 시각화합니다."""
    assert n_rows * n_cols == n_heads

    font_path = './NanumBarunGothic.ttf'
    font_prop = fm.FontProperties(fname=font_path, size=8)

    fig = plt.figure(figsize=(12, 28))  # x축을 조금 넓혀서 압축 줄임 (10->12)

    # 번역된 문장과 원본 문장을 토큰 단위로 분리
    sentence_tokens = sentence.split()
    translation_tokens = translation.split()

    for i in range(n_heads):
        ax = fig.add_subplot(n_rows, n_cols, i + 1)

        # attention shape: (head_idx, tgt_len, src_len)
        _attention = attention.squeeze(0)[i].cpu().detach().numpy()

        # extent 명시: (-0.5, src_len-0.5, tgt_len-0.5, -0.5)로 ticks와 맞춤
        src_len = len(sentence_tokens)
        tgt_len = len(translation_tokens)
        cax = ax.matshow(_attention, cmap='viridis', extent=[-0.5, src_len - 0.5, tgt_len - 0.5, -0.5])

        # 눈금 위치 설정
        ax.set_xticks(range(src_len))
        ax.set_yticks(range(tgt_len))

        # 라벨 설정: ha/va로 중앙 정렬
        # 다른분꺼 보니까 45도가 좋아보이더만
        ax.set_xticklabels(sentence_tokens, rotation=45, fontproperties=font_prop, ha='center', va='center')
        ax.set_yticklabels(translation_tokens, fontproperties=font_prop, ha='right', va='center')

        ax.tick_params(labelsize=8, pad=15)  # pad로 텍스트와 tick 간격 미세 조정

    plt.tight_layout()  # subplot 간 여백 자동 조정 (밀림 방지)
    plt.show()

display_attention(src, beam_translation, beam_attention[-1])

## 10. 최종 모델 성능 종합 평가

In [ ]:
nltk.download('wordnet')

from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
from bert_score import score as bert_scorer
from tqdm.notebook import tqdm
import numpy as np

def evaluate_model_comprehensively(model, src_corpus, tgt_corpus, src_tokenizer, tgt_tokenizer, device, translate_function, beam_size=5):
    """
    테스트 데이터셋 전체에 대해 번역을 수행하고,
    BLEU, METEOR, ROUGE, BERTScore를 포함한
    종합적인 평가지표를 계산하여 출력
    """
    model.eval()

    # 1. 전체 테스트 데이터셋에 대해 번역 생성
    print("테스트 데이터셋 전체에 대한 번역을 시작합니다...")
    predictions = []
    references = []

    for src_sentence, ref_sentence in tqdm(zip(src_corpus, tgt_corpus), total=len(src_corpus), desc="Translating"):
        pred_sentence, _ = translate_function(
            src_sentence, src_tokenizer, tgt_tokenizer, model, device, beam_size=beam_size
        )
        predictions.append(pred_sentence)
        references.append(ref_sentence)

    print("번역 완료. 평가지표 계산을 시작합니다...")

    # 2. N-gram 기반 평가지표 계산 (BLEU, METEOR, ROUGE)
    print("Calculating BLEU, METEOR, ROUGE scores...")
    pred_tokens = [p.split() for p in predictions]
    ref_tokens = [[r.split()] for r in references]

    # BLEU
    smooth_fn = SmoothingFunction().method1
    bleu_score = np.mean([sentence_bleu(r, p, smoothing_function=smooth_fn) for r, p in zip(ref_tokens, pred_tokens)])

    # METEOR
    meteor_score_avg = np.mean([meteor_score(r, p) for r, p in zip(ref_tokens, pred_tokens)])

    # ROUGE
    rouge_calculator = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    rouge1_f1, rougeL_f1 = [], []
    for ref, pred in zip(references, predictions):
        scores = rouge_calculator.score(ref, pred)
        rouge1_f1.append(scores['rouge1'].fmeasure)
        rougeL_f1.append(scores['rougeL'].fmeasure)
    rouge1_avg = np.mean(rouge1_f1)
    rougeL_avg = np.mean(rougeL_f1)

    # 3. 의미 기반 평가지표 계산 (BERTScore)
    print("Calculating BERTScore...")
    P, R, F1 = bert_scorer(predictions, references, lang="en", device=device, verbose=True)
    bert_f1_score = F1.mean().item()

    # 4. 최종 결과 종합 출력
    print("\n" + "="*40)
    print("      종합 번역 성능 평가 결과      ")
    print("="*40)
    print(f"  BLEU Score   : {bleu_score * 100:.2f}")
    print(f"  METEOR Score : {meteor_score_avg * 100:.2f}")
    print(f"  ROUGE-1 (F1) : {rouge1_avg * 100:.2f}")
    print(f"  ROUGE-L (F1) : {rougeL_avg * 100:.2f}")
    print(f"  BERTScore (F1): {bert_f1_score * 100:.2f}")
    print("="*40)

# 함수 호출하여 종합 평가 실행
# 이전에 로드한 inference_model과 Mecab 처리된 test_kor_mecab을 사용
evaluate_model_comprehensively(
    inference_model,
    test_kor_mecab,
    test_eng_corpus,
    ko_tokenizer,
    en_tokenizer,
    device,
    translate_sentence_beam_search, # 빔 서치 함수 사용
    beam_size=5
)
